In [ ]:
# !mkdir ~/.kaggle

In [ ]:
# !cp /content/drive/MyDrive/GAN_Datasets/kaggle.json ~/.kaggle/

In [ ]:
# !kaggle datasets download -d hugodarwood/epirecipn

In [ ]:
# !unzip /content/epirecipes.zip -d /content/drive/MyDrive/GAN_Datasets/Recipes/

In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import json
import re
import string

import tensorflow as tf
from tensorflow.keras import layers, models, callbacks, losses

In [ ]:
VOCAB_SIZE = 10000
MAX_LEN = 200
EMBEDDING_DIM = 100
N_UNITS = 128
VALIDATION_SPLIT = 0.2
SEED = 42
LOAD_MODEL = False
BATCH_SIZE = 32
EPOCHS = 25

# Load the data

In [ ]:
with open("/content/drive/MyDrive/GAN_Datasets/Recipes/full_format_recipes.json") as json_data:
    recipe_data = json.load(json_data)

In [ ]:
# Filter the dataset
filtered_data = [
    "Recipe for " + x["title"] + " | " + " ".join(x["directions"])
    for x in recipe_data
    if "title" in x
    and x["title"] is not None
    and "directions" in x
    and x["directions"] is not None
]

In [ ]:
# Count the recipes
n_recipes = len(filtered_data)
print(f"{n_recipes} recipes loaded")

20111 recipes loaded


In [ ]:
example = filtered_data[9]
print(example)

Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas  | Chop enough parsley leaves to measure 1 tablespoon; reserve. Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan, covered, 5 minutes. Meanwhile, sprinkle gelatin over water in a medium bowl and let soften 1 minute. Strain broth through a fine-mesh sieve into bowl with gelatin and stir to dissolve. Season with salt and pepper. Set bowl in an ice bath and cool to room temperature, stirring. Toss ham with reserved parsley and divide among jars. Pour gelatin on top and chill until set, at least 1 hour. Whisk together mayonnaise, mustard, vinegar, 1/4 teaspoon salt, and 1/4 teaspoon pepper in a large bowl. Stir in celery, cornichons, and potatoes. Pulse peas with marjoram, oil, 1/2 teaspoon pepper, and 1/4 teaspoon salt in a food processor to a coarse mash. Layer peas, then potato salad, over ham.


 # Tokenize the data

In [ ]:
# Pad the punctuation, to treat them as separate 'words'
def pad_punctuation(s):
    s = re.sub(f"([{string.punctuation}])", r" \1 ", s)
    s = re.sub(" +", " ", s)
    return s

text_data = [pad_punctuation(x) for x in filtered_data]

In [ ]:
# Display an example of a recipe
example_data = text_data[9]
example_data

'Recipe for Ham Persillade with Mustard Potato Salad and Mashed Peas | Chop enough parsley leaves to measure 1 tablespoon ; reserve . Chop remaining leaves and stems and simmer with broth and garlic in a small saucepan , covered , 5 minutes . Meanwhile , sprinkle gelatin over water in a medium bowl and let soften 1 minute . Strain broth through a fine - mesh sieve into bowl with gelatin and stir to dissolve . Season with salt and pepper . Set bowl in an ice bath and cool to room temperature , stirring . Toss ham with reserved parsley and divide among jars . Pour gelatin on top and chill until set , at least 1 hour . Whisk together mayonnaise , mustard , vinegar , 1 / 4 teaspoon salt , and 1 / 4 teaspoon pepper in a large bowl . Stir in celery , cornichons , and potatoes . Pulse peas with marjoram , oil , 1 / 2 teaspoon pepper , and 1 / 4 teaspoon salt in a food processor to a coarse mash . Layer peas , then potato salad , over ham . '

In [ ]:
text_ds = tf.data.Dataset.from_tensor_slices(text_data).batch(BATCH_SIZE).shuffle(1000)

In [ ]:
# Create Text vectorization
vectorize_layer = layers.TextVectorization(standardize='lower', max_tokens=VOCAB_SIZE,
                                           output_mode='int', output_sequence_length= MAX_LEN +1 )

In [ ]:
# Adapt the layer to the training set
vectorize_layer.adapt(text_ds)
vocab = vectorize_layer.get_vocabulary()

In [ ]:
# Display some token:word mappings
for i, word in enumerate(vocab[:10]):
    print(f"{i}: {word}")

0: 
1: [UNK]
2: .
3: ,
4: and
5: to
6: in
7: the
8: with
9: a


In [ ]:
# Display the same example converted to ints
example_tokenised = vectorize_layer(example_data)
print(example_tokenised.numpy())

[  26   16  557    1    8  298  335  189    4 1054  494   27  332  228
  235  262    5  594   11  133   22  311    2  332   45  262    4  671
    4   70    8  171    4   81    6    9   65   80    3  121    3   59
   12    2  299    3   88  650   20   39    6    9   29   21    4   67
  529   11  164    2  320  171  102    9  374   13  643  306   25   21
    8  650    4   42    5  931    2   63    8   24    4   33    2  114
   21    6  178  181 1245    4   60    5  140  112    3   48    2  117
  557    8  285  235    4  200  292  980    2  107  650   28   72    4
  108   10  114    3   57  204   11  172    2   73  110  482    3  298
    3  190    3   11   23   32  142   24    3    4   11   23   32  142
   33    6    9   30   21    2   42    6  353    3 3224    3    4  150
    2  437  494    8 1281    3   37    3   11   23   15  142   33    3
    4   11   23   32  142   24    6    9  291  188    5    9  412  572
    2  230  494    3   46  335  189    3   20  557    2    0    0    0
    0 

# Create the Training Set

In [ ]:
# Create the training set of recipes and the same text shifted by one word
def prepare_inputs(text):
    text = tf.expand_dims(text, -1)
    tokenized_sentences = vectorize_layer(text)
    x = tokenized_sentences[:, :-1]
    y = tokenized_sentences[:, 1:]
    return x, y

train_ds = text_ds.map(prepare_inputs)

 # Build the LSTM

In [ ]:
inputs = layers.Input(shape=(None,), dtype='int32')
x = layers.Embedding(VOCAB_SIZE, EMBEDDING_DIM)(inputs)
x = layers.LSTM(N_UNITS, return_sequences=True)(x)
outputs = layers.Dense(VOCAB_SIZE, activation='softmax')(x)
lstm = models.Model(inputs, outputs)
lstm.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 100)         1000000   
                                                                 
 lstm (LSTM)                 (None, None, 128)         117248    
                                                                 
 dense (Dense)               (None, None, 10000)       1290000   
                                                                 
Total params: 2407248 (9.18 MB)
Trainable params: 2407248 (9.18 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
if LOAD_MODEL:
  lstm = models.load_model("./models/lstm", compile=False)

## Train the LSTM

In [ ]:
loss_fn = losses.SparseCategoricalCrossentropy()
lstm.compile("adam", loss_fn)

In [ ]:
# Create a TextGenerator checkpoint
class TextGenerator(callbacks.Callback):
  def __init__(self, index_to_word, top_k=10):
    self.index_to_word = index_to_word
    self.word_to_index = {word:index for index, word in enumerate(index_to_word)}

  def sample_from(self, probs, temperature):
    probs = probs ** (1/temperature)
    probs = probs/np.sum(probs)
    return np.random.choice(len(probs),p=probs), probs

  def generate(self, start_prompt, max_tokens, temperature):
    # print(f"start_prompt:{start_prompt}")
    start_tokens = [self.word_to_index.get(x,1)  for x in start_prompt.split()]
    sample_token = None
    info = []
    while len(start_tokens) < max_tokens and sample_token != 0:
      x = np.array([start_tokens])
      y = self.model.predict(x, verbose=0)
      # print(f"y:{y.shape}")
      sample_token, probs = self.sample_from(y[0][-1], temperature)
      info.append({"prompt": start_prompt, "word_probs": probs})
      start_tokens.append(sample_token)
      start_prompt = start_prompt + " " + self.index_to_word[sample_token]
    print(f"\ngenerated text:\n{start_prompt}\n")
    return info

  def on_epoch_end(self, epoch, logs=None):
    self.generate("recipe for", max_tokens=100, temperature=1.0)

In [ ]:
# create a model save checkpoint
model_checkpoint_callback = callbacks.ModelCheckpoint(
    filepath = "./checkpoint/checkpoint.ckpt",
    save_weights_only=True,
    save_freq ="epoch",
    verbose = 0
)

tensor_callback = callbacks.TensorBoard(log_dir="./logs")

# Tokenize starting prompt
text_generator = TextGenerator(vocab)

In [ ]:
lstm.fit(train_ds, epochs= EPOCHS, callbacks=[model_checkpoint_callback,tensor_callback, text_generator])

Epoch 1/25
628/629 [============================>.] - ETA: 0s - loss: 4.4430
generated text:
recipe for curried cherries garbanzo grilled mango coriander | , oven 2 soufflé oil , | cook sugar in medium heat , 2 until low food freezer for 15 bowl with 2 to skillet , no butter sage . cover and salt ( a forms from turning together hours on beat cranberries . 5 , about chop skewers to medium of 8 to into chill to airtight thick and high to fill the maker , discarding stir intact to colander . separate if large aioli . shellfish is medium - prepared rice under on wiggling the to pine not 

629/629 [==============================] - 73s 110ms/step - loss: 4.4416
Epoch 2/25
628/629 [============================>.] - ETA: 0s - loss: 3.1070
generated text:
recipe for starting apple tenderloin | place all to wide an pot of bowl to heat . add tomatoes and stir in boiling heat . add more , and sauté until cold . add red salt and pepper and armagnac to hot 20 minutes . ) enough hot this of surface 

In [ ]:
lstm.save("./models/lstm/")

# Generate text using LSTM

In [ ]:
def print_probs(info, vocab, top_k=5):
  for i in info:
    print(f"\nPROMPT: {i['prompt']}")
    word_probs = i["word_probs"]
    p_sorted = np.sort(word_probs)[::-1][:top_k]
    i_sorted = np.argsort(word_probs)[::-1][:top_k]
    for p, i in zip(p_sorted, i_sorted):
            print(f"{vocab[i]}:   \t{np.round(100*p,2)}%")
    print("--------\n")

In [ ]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 /", max_tokens=20, temperature=1.0
)


generated text:
recipe for roasted vegetables | chop 1 / 4 pound kernels with a mortar and pestle or in a food



In [ ]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 /
4:   	67.41%
2:   	16.98%
3:   	10.17%
8:   	1.28%
1:   	1.06%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4
cup:   	52.22%
teaspoon:   	16.18%
-:   	12.43%
tsp:   	6.56%
.:   	1.6%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4 pound
zest:   	6.01%
garlic:   	4.34%
chicken:   	3.43%
butter:   	3.37%
fennel:   	2.99%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4 pound kernels
from:   	39.77%
and:   	20.96%
.:   	10.15%
with:   	7.76%
,:   	6.37%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4 pound kernels with
a:   	62.52%
1:   	4.02%
the:   	2.95%
garlic:   	2.35%
mortar:   	1.95%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4 pound kernels with a
mortar:   	37.64%
fork:   	16.39%
sharp:   	8.91%
knife:   	7.12%
slotted:   	4.57%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4 pound kernels with a mortar
and:   	89.79%
,:   	2.0%
medium:   	1.47%

In [ ]:
info = text_generator.generate(
    "recipe for roasted vegetables | chop 1 /", max_tokens=10, temperature=0.2
)


generated text:
recipe for roasted vegetables | chop 1 / 4 cup



In [ ]:
print_probs(info, vocab)


PROMPT: recipe for roasted vegetables | chop 1 /
4:   	99.89%
2:   	0.1%
3:   	0.01%
8:   	0.0%
1:   	0.0%
--------


PROMPT: recipe for roasted vegetables | chop 1 / 4
cup:   	99.64%
teaspoon:   	0.28%
-:   	0.08%
tsp:   	0.0%
.:   	0.0%
--------

